#### Projeto TCGA - LGG Normal vs Maligno ####

Objetivo: análise comparativa de dados genômicos e clínicos de amostras normais e malignas disponíveis no banco de dados do National Cancer Institute (TCGA - The Cancer Genome Atlas) do câncer de mama (TCGA-BRCA), câncer de pulmão (TCGA-LUAD) e câncer de intestino (TCGA-COAD). 
O foco será em: vias de sinalização do câncer (Gene Set Enrichment Analysis), expressão gênica (RNA-seq/STAR counts), genes diferencialmente expressos e dados clínicos.
- Comparar o perfil de mutações entre amostras normais e malignas.
    - Identificar o padrão de heterogenidade do tumor (a partir do padrão de mutação)
- Analisar a expressão diferencial de genes entre as amostras.
- Identificar potenciais marcadores moleculares associados ao fenótipo maligno.
    - A partir da análise de expressão diferencial e mutações, selecione os genes com maior potencial de serem marcadores moleculares, realizando uma análise de enriquecimento de caminho (pathway enrichment) para entender os processos biológicos envolvidos.
- Correlacionar os dados clínicos com os achados genômicos.

#### Pré-processamento dos dados e análise descritiva ####
- Manifesto 
- Expressão gênica (RNA-seq/STAR counts)
    - Processamento de dados RNA-seq alinhando os reads com um genoma de referência e normalizando os dados para contagem (STAR counts). 
    - Verifique e trate valores ausentes ou discrepantes e normalize para facilitar comparações entre amostras.

- Dados Clínicos
    - Baixar os dados clínicos {OK}
    - Alinhar as infrmações com as amostras do TCGA
    - Verificar as variáveis importantes {filtrar melhor as colunas}


#### Análise Descritiva ####
- Analise de Mutações
    - Tabelas de contagem e gráficos de barras para mostrar a frequência de mutações em genes relevantes para câncer
    - Diferenciar as amostras normais e malignas 

- Expressão gênica
    - Gráficos de distribuição (boxplots) comparando a expressão gênica nas normais e malignas 
    - Normalização dos dados (usando TPM ou FPKM)
    - Use mapas de calor para ilustrar a expressão gênica entre as amostras, destacando os genes diferencialmente expressos

- Análise Clínica:
    - Tabela descritiva com as principais variáveis clínicas {selecionar as variáveis}
    - histogramas para visualizar a distribuição


#### Análise Exploratória ####
- Perfil de Mutações: 
    - Utilize ferramentas de bioinformática, como MutSigCV, para identificar genes que apresentam mutações significativas associadas ao fenótipo maligno. 
    - Aplique testes estatísticos, como o teste qui-quadrado, para comparar a frequência de mutações entre amostras normais e malignas.

- Expressão Diferencial: 
    - Use pacotes como DESeq2 ou edgeR para realizar a análise de expressão diferencial de genes. 
    - Compare as expressões dos genes entre as amostras normais e malignas, utilizando testes de hipótese (como t-test ou ANOVA).

- Correlação com Dados Clínicos: 
    - Realize uma análise de correlação entre os dados genômicos e clínicos usando métodos como análise de regressão linear, análise de sobrevivência de Kaplan-Meier ou regressão de Cox para identificar fatores associados ao prognóstico clínico.

### Código no R ###
#Para correlacionar os dados clínicos 
####Instalação dos pacotes
if (!require("BiocManager", quietly = TRUE))
  install.packages("BiocManager")
if (!require("data.table", quietly = TRUE))
  install.packages("BiocManager")
if (!require("DESeq2", quietly = TRUE))
  BiocManager::install("DESeq2")

#Instalar pacotes Bioconductor
BiocManager::install(c("TCGAbiolinks", "SummarizedExperiment"))

#Instalar pacotes CRAN
install.packages(c("survminer", "survival", "tidyverse"))

###bibliotecas 
library(TCGAbiolinks)
library(survminer)
library(SummarizedExperiment)
library(tidyverse)
library(data.table)
library(DESeq2)
library(dplyr)

#getting clinical data for TCGA-BRCA, TCGA-LUAD and TCGA-COAD cohort
clinical_brca <- GDCquery_clinic("TCGA-BRCA")
clinical_luad <- GDCquery_clinic("TCGA-LUAD")
clinical_coad <- GDCquery_clinic("TCGA-COAD")
#filtrar as colunas 
columns_to_keep <- c("project", 
                     "submitter_id", 
                     "tissue_or_organ_of_origin", 
                     "tumor_grade", 
                     "ajcc_pathologic_t", 
                     "classification_of_tumor", 
                     "diagnosis_id", 
                     "gender", 
                     "ethnicity", 
                     "age_at_index", 
                     "vital_status", 
                     "bcr_patient_barcode")

clinical_brca_filtered <- clinical_brca %>%
  select(all_of(columns_to_keep))

clinical_luad_filtered <- clinical_luad %>%
  select(all_of(columns_to_keep))

clinical_coad_filtered <- clinical_coad %>%
  select(all_of(columns_to_keep))

#Combinar os dados em uma única tabela
clinical_data_combined <- bind_rows(
  clinical_brca_filtered,
  clinical_luad_filtered,
  clinical_coad_filtered
)


#Carregar o manifesto
manifesto <- read.delim("C:/Users/Usuário/Downloads/Bioinformatica/LUAD/gdc_manifest.BRCA_LUAD_COAD.txt.map2submitterID", 
                        header = FALSE, sep = "\t")

#Renomear as colunas para facilitar o trabalho
manifesto <- manifesto[-1, ]
colnames(manifesto) <- c(
  "case_id", "aliquot_id", "submitter_id_full", "sample_id", "sample_type",
  "bcr_patient_barcode", "tissue_type", "case_submitter_id", "data_category",
  "data_type", "file_id", "file_name", "id"
)
#Unir as tabelas {não deu certo}
#merged_data <- merge(clinical_data_combined, manifesto, by = "bcr_patient_barcode", all.x = TRUE)

#Padronizar e remover sufixos para obter IDs de pacientes
manifesto$patient_id <- sub("-[0-9][0-9][A-Z]*$", "", manifesto$submitter_id)

Testar:
#Identificar pacientes com amostras tumorais e normais
tumor_samples <- manifesto[manifesto$sample_type == "Primary Tumor", "patient_id"]
normal_samples <- manifesto[manifesto$sample_type == "Solid Tissue Normal", "patient_id"]

#Obter interseção de pacientes com ambos os tipos
patients_both <- intersect(tumor_samples, normal_samples)
length(patients_both) # Número de pacientes com amostras tumorais e normais
#Filtrar dados clínicos
clinical_filtered <- clinical_data_combined[clinical_data_combined$bcr_patient_barcode %in% patients_both, ]

#Filtrar manifesto
manifesto_filtered <- manifesto[manifesto$patient_id %in% patients_both, ]

## Criar a Matriz de Expressão

#Obter apenas arquivos relevantes (exemplo: tumor e normal)
expression_files <- manifesto_filtered$file_name

#Baixar os dados de exp gênica

#Caminho completo para os arquivos (ajuste conforme necessário)
file_paths <- file.path("caminho/para/dados/expressao", expression_files)

#Ler arquivos de expressão e criar matriz (exemplo para arquivos TSV)
library(data.table)
expression_list <- lapply(file_paths, fread)  # Ler todos os arquivos como data.tables
names(expression_list) <- manifesto_filtered$submitter_id  # Nomear pelas amostras

#Combinar em uma única matriz por gene
expression_matrix <- Reduce(function(x, y) merge(x, y, by = "gene_id"), expression_list)

#Ajustar colunas e nomes (substituir IDs por pacientes, por exemplo)
colnames(expression_matrix)[-1] <- manifesto_filtered$patient_id


#Identificar quais colunas correspondem a tumor e normal
tumor_columns <- manifesto_filtered[manifesto_filtered$sample_type == "Primary Tumor", "patient_id"]
normal_columns <- manifesto_filtered[manifesto_filtered$sample_type == "Solid Tissue Normal", "patient_id"]

#Matriz final (genes x pacientes com tumor e normal)
expression_tumor <- expression_matrix[, tumor_columns, drop = FALSE]
expression_normal <- expression_matrix[, normal_columns, drop = FALSE]

#Pacientes finais
final_patients <- intersect(colnames(expression_tumor), colnames(expression_normal))
expression_final <- expression_matrix[, c(final_patients, final_patients), drop = FALSE]



#### 